In [1]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


'\nPurpose: To decompose the multi-somas for splitting\nusing the new decomposition method\n\n\n\n'

In [15]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

In [18]:
test_mode = False

In [38]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-13 08:18:55,655 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-13 08:18:55,656 - settings - Setting database.user to celiib
INFO - 2021-01-13 08:18:55,657 - settings - Setting database.password to newceliipass
INFO - 2021-01-13 08:18:55,662 - settings - Setting enable_python_native_blobs to True


Sleeping 100 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-13 08:18:55,968 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [30]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [31]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 4130


In [32]:
#so that it will have the adapter defined
from datajoint_utils import *

In [33]:
decimation_version = 0
decimation_ratio = 0.25
key_source = (minnie.Decimation().proj(decimation_version='version')  & 
                  dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
                  & minnie.MultiSomaProofread2() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid()).proj()))
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134947393276,0,0.25
864691135012398582,0,0.25
864691135012471798,0,0.25
864691135012571126,0,0.25
864691135012905110,0,0.25
864691135065018692,0,0.25
864691135065024068,0,0.25
864691135065032772,0,0.25
864691135065033540,0,0.25
864691135081745143,0,0.25


In [34]:
#schema.external['decomposition'].delete(delete_external_files=True)

In [35]:
#minnie.Decomposition.drop()

In [40]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25
process_version = 0

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    process_version : int unsigned #the version of the preprocessing pipeline run
    index : tinyint unsigned  #the index of the neuron object that resulted from that mesh (indexed starting at 0)
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """


    
    key_source = (minnie.Decimation().proj(decimation_version='version')  & 
                  dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
                  & minnie.MultiSomaProofread2() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid()).proj()))
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)
        

        #3) Get the somas info 
        somas = du.get_soma_mesh_list(segment_id) 
        soma_ver = du.get_soma_mesh_list_ver(segment_id)
                  
        
        print(f"somas = {somas}")
                  
        #3b) Get the glia and nuclei information 
        glia_faces,nuclei_faces = du.get_segment_glia_nuclei_faces(segment_id,return_empty_list=True)
        
                  
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"],
        glia_faces=glia_faces,
        nuclei_faces = nuclei_faces,

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       ver = soma_ver,
                       process_version = process_version,
                       index = 0,
                       multiplicity=1,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [41]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

INFO - 2021-01-13 08:19:21,122 - autopopulate - Found 143 keys to populate
INFO - 2021-01-13 08:19:21,132 - connection - Transaction started
INFO - 2021-01-13 08:19:21,134 - autopopulate - Populating: {'segment_id': 864691134947393276, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started



---- Working on Neuron 864691134947393276 ----
somas = [[<trimesh.Trimesh(vertices.shape=(19028, 3), faces.shape=(37767, 3))>], array([547.5166]), array([0.818])]

--- Beginning preprocessing of 864691134947393276---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(19028, 3), faces.shape=(37767, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(19028, 3), faces.shape=(37767, 3))>]
soma_mesh_list_centers = [array([604782.36627076, 817703.83371873, 923126.39603741])]
Getting Glia and Nuclei Pieces Subtracted Away 1.0773143768310547
 Splitting mesh after soma cancellation 1.4604737758636475
# of split_meshes = 19
 Containing Mesh Indices 0.04474449157714844
 non_soma_touching_meshes 2.1696090698242188e-05
There were 18 pieces found after size threshold
 Finding inside pieces and no

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:09<00:00,  9.45s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 11.91020131111145
connecting at the root
branches_touching_root = [98]
length of Graph = 36893
Working on path [3129. 3161. 3184. 3208. 3226. 3246. 3264.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [36893.  3315.  3349.  3384.  3418.  3452.  3490.  3502.]
path_degrees = [4, 2, 2, 2, 2, 2, 2, 3]
Working on path [36894.  3517.  3536.  3565.  3593.  3634.  3645.  3650.]
path_degrees = [5, 2, 2, 2, 2, 2, 2, 3]
Working on path [24841. 24843. 24848. 24850. 24855. 24857.]
path_degrees = [3, 2, 2, 2, 2, 3]
Working on path [28920. 28946. 28972. 28999. 29027. 29029.]
path_degrees = [3, 2, 2, 2, 2, 3]
Working on path [30188. 30215. 30242. 30265. 30274.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [31389. 31403. 31417. 31432. 31446.]
path_degrees = [3, 2, 2, 2, 3]
After combining close endpoints max(kept_branches_idx) = 98, len(kept_branches_idx) = 92
empty_indices % = 0.0
 conflict_indices % = 0.005023365876131107


...


Deleting branches from dictionary
MP_branches_with_stitch_point= [0]
MP_branches_for_correspondence = [0]
MP_leftover = [], MP_leftover_idx = []
 Finished with (0, 0) 



Time for decomp of Limb = 40.53995180130005
Number of matching vertices = 9
Only one endpoint after filtering away the endpoints that are not on the skeleton
Using Distance measure skeletal_distance
endpoints_must_keep = [[613865. 814225. 923637.]]
Number of end_nodes BEFORE filtering = 4
all_single_nodes_to_eliminate = [0]



Removed 1 skeletal branches



41569 41570
For marked faces: None
empty_indices % = 0.0
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test



Total time for Skeletonization and Mesh Correspondence = 930.5024933815002


 ----- Working on Stitching ----------
Status of Main limb stitch point moved = False
Status of Main limb stitch point moved = True
The closest float distance was 19734.828848609515 which was greater than the maximum stitch distance 8000
 --> so ending the floating mesh stitch processs
Total time for stitching floating pieces = 9.858852624893188
Total time for Concept Networks = 9.362931728363037
Total time for all mesh and skeletonization decomp = 949.7247850894928
No soma volume ratios so computing them now
Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 4130 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_402444.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_402444_poisson.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/poisson_912860.mls
re